In [25]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()  # 기본 버킷
prefix = "bill-lstm3"
role = get_execution_role()

# S3에 업로드
model_path = sagemaker_session.upload_data(
    path="/home/ec2-user/SageMaker/deploy/lstm_bill/bill_model.tar.gz",
    bucket=bucket,
    key_prefix=prefix
)

print("✅ S3 업로드 완료:", model_path)

✅ S3 업로드 완료: s3://sagemaker-ap-northeast-2-471112588210/bill-lstm3/bill_model.tar.gz


In [27]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

model = PyTorchModel(
    model_data=model_path,
    role=role,
    entry_point='inference.py',
    source_dir='/home/ec2-user/SageMaker/deploy/lstm_bill',
    framework_version='1.13.1',
    py_version='py39'
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type= 'ml.m5.xlarge',
    endpoint_name='bill-lstm-endpoint-v9',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)


[05/30/25 15:00:22] INFO     Repacking model artifact                                                  ]8;id=491135;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=409995;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/model.py#820\820]8;;\
                             (s3://sagemaker-ap-northeast-2-471112588210/bill-lstm3/bill_model.tar.gz)             
                             , script artifact (/home/ec2-user/SageMaker/deploy/lstm_bill), and                    
                             dependencies ([]) into single tar.gz file located at                                  
                             s3://sagemaker-ap-northeast-2-471112588210/pytorch-inference-2025-05-30-1             
                             5-00-22-176/model.tar.gz. This may take some time depending on model                  
                             size...                                                                               

                    INFO     Creating model with name: pytorch-inference-2025-05-30-15-00-22-772    ]8;id=765952;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=910818;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[05/30/25 15:00:23] INFO     Creating endpoint-config with name bill-lstm-endpoint-v9               ]8;id=398512;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=541502;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\

                    INFO     Creating endpoint with name bill-lstm-endpoint-v9                      ]8;id=626558;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=934623;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

-------!

In [ ]:
import pandas as pd

# 추론 입력 (timestamp 포함 필수)
df = pd.read_csv("inference_input.csv")
payload = {
    "data": df.to_dict(orient="records"),
    "horizon": 21   # 예측 길이 (예: 5월 10일~31일 → 21시간/일)
}

response = predictor.predict(payload)
print(response)


In [26]:
import boto3

sm = boto3.client('sagemaker')

# 삭제할 엔드포인트 이름 입력
endpoint_name = 'bill-lstm-endpoint-v8'

# 엔드포인트 삭제
sm.delete_endpoint(EndpointName=endpoint_name)

print(f"✅ 엔드포인트 '{endpoint_name}' 삭제 요청 완료")


✅ 엔드포인트 'bill-lstm-endpoint-v8' 삭제 요청 완료


In [9]:
# 엔드포인트 삭제
!aws sagemaker delete-endpoint --endpoint-name bill-lstm-endpoint-v3

# 필요시 엔드포인트 구성도 삭제
# !aws sagemaker delete-endpoint-config --endpoint-config-name bill-lstm-endpoint-v3



An error occurred (ValidationException) when calling the DeleteEndpoint operation: Cannot update in-progress endpoint "arn:aws:sagemaker:ap-northeast-2:471112588210:endpoint/bill-lstm-endpoint-v3".
